In [2]:
%matplotlib inline

In [6]:
from __future__ import division
import numpy as np
from vtk_rw import read_vtk, write_vtk
import matplotlib.pyplot as plt
from joblib import Memory
from tempfile import mkdtemp

In [19]:
cachedir = mkdtemp()
memory = Memory(cachedir=cachedir, mmap_mode='r')

## Sample T1 data (average / profiles) on simple surface

In [36]:
label_file = ('/scr/ilz3/myelinconnect/groupavg/indv_space/BP4T/labelling_lh.txt')
highres_file = ('/scr/ilz3/myelinconnect/struct/surf_rh/prep_t1/profiles/BP4T_rh_mid_proflies.vtk')
old_lowres_file = ('/scr/ilz3/myelinconnect/groupavg/indv_space/BP4T/lowres_lh_d_def.vtk')
new_lowres_file = ('/scr/ilz3/myelinconnect/final_data_on_surfaces/t1/BP4T_lh_profiles_lowres.vtk')

In [34]:
labels = np.loadtxt(label_file)

In [37]:
highres_v, highres_f, highres_data = read_vtk(highres_file)

In [32]:
lowres_v, lowres_f, lowres_data = read_vtk(old_lowres_file)

In [20]:
@memory.cache
def combine_patch_data(labels, highres_data):
    
    lowres_data = np.empty((int(labels[:,1].max()+1), highres_data.shape[1]))
   
    for l in range(int(labels[:,1 ].max())):
        patch = np.where(labels[:,1]==l)[0]
        patch_data = highres_data[patch]
        patch_mean = np.mean(patch_data, axis=0)
        lowres_data[l] = patch_mean
    
    return lowres_data

In [38]:
lowres_data = combine_patch_data(labels, highres_data)

________________________________________________________________________________
[Memory] Calling __main__--home-raid3-huntenburg-workspace-myelinconnect-notebooks-__ipython-input__.combine_patch_data-alias...
combine_patch_data-alias(array([[  0.000000e+00,   6.300000e+01],
       ..., 
       [  7.371470e+05,   7.618000e+04]]), 
array([[ 2074.838867, ...,  1974.813477],
       ..., 
       [ 1572.972046, ...,  2181.115479]]))
________________________________________combine_patch_data-alias - 77.3s, 1.3min


In [26]:
np.asarray(lowres_data)

array([[  1.52529907e+003,   1.60783521e+003,   1.54792151e+003, ...,
          1.75336914e+003,   1.80376611e+003,   1.97734473e+003],
       [  1.54620874e+003,   1.71543667e+003,   2.10014530e+003, ...,
          1.94961078e+003,   2.04899866e+003,   2.02068624e+003],
       [  1.50893168e+003,   1.62895601e+003,   1.75345016e+003, ...,
          1.93022871e+003,   1.91933935e+003,   2.04546982e+003],
       ..., 
       [  1.56051676e+003,   1.59922291e+003,   1.64735329e+003, ...,
          1.82436155e+003,   1.93492713e+003,   2.04554342e+003],
       [  1.59857990e+003,   1.63113521e+003,   1.67141414e+003, ...,
          1.82459194e+003,   1.93498773e+003,   2.09649526e+003],
       [  5.46228828e-315,   5.46228856e-315,   5.46228883e-315, ...,
          9.06498498e-043,   6.12388154e-062,   3.31253229e-317]])

In [ ]:
write_vtk(new_lowres_file, lowres_v, lowres_f, data=lowres_data)